## imports

In [1]:
import re
import json

alphanumeric='abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789'
dot =  '('+('|'.join(alphanumeric))+')'

# print(dot)


# Regex to Postfix

In [3]:
def validate_regex(regex):
    try:
       re.compile(regex)
       return True
    except:
       return False 
    



def handle_dot(regex):
   return re.sub(r'\.', dot, regex)

def handle_range(regex):
    new_regex = ""
    replace = False
    start = None
    end= None
    for i in range(len(regex)):
        if regex[i] == '[':
            new_regex += '('
            replace =True
        elif regex[i]== ']':
             new_regex += ')'
             replace =False
        elif regex[i] == '-':
            replace = True
            start = alphanumeric.index(regex[i-1])
            end = alphanumeric.index(regex[i+1])
            for j in range(start+1,end):
                new_regex += alphanumeric[j]
                if(replace and regex[i+1] != ']'):
                    new_regex+='|'
        else:
           new_regex += regex[i] 
           if(replace and regex[i+1] != ']'):
                    new_regex+='|'


    return new_regex  


def concatenate(regex):
    new_regex = ""
    for i in range(len(regex)-1):
        new_regex += regex[i] 
        if regex[i] in ['*','+','?',')',']'] and regex[i+1] not in ['*','+','?',')',']','.', '|']:
            new_regex+="."
        elif regex[i] in alphanumeric and  (regex[i+1] in alphanumeric or regex[i+1] in ['(','[']):
            new_regex+="."
    
    new_regex += regex[-1]

    return new_regex

def infix_to_postfix(regex):
    queue = []
    stack = []
    operators={'(':0,"|":1, ".":2, "?":3, "+":4, "*":5}

    for i in range(len(regex)):
        if regex[i] == '(':
            stack.append(regex[i])
        elif regex[i] == ')':
            while stack[-1] != "(":  
                queue.append(stack.pop()) 
            stack.pop()     
        elif regex[i] in operators:
            while stack  and operators[regex[i]] <= operators[stack[-1]]:
                queue.append(stack.pop())
            stack.append(regex[i])

        else :
            queue.append(regex[i])    
    
    while stack:
        queue.append(stack.pop())

    return ''.join(queue  )  


def to_postfix(regex):
    if(validate_regex(regex)):
       regex = handle_dot(regex)
       print(regex)
       regex = handle_range(regex)
       print(regex)
       regex = concatenate(regex)
       print(regex)
       regex = infix_to_postfix(regex)
       print(regex)

       return regex


    else:
        print('sorry you have entered invalid regex! ')    


    

# print(handle_dot('ab.c'))
# print(handle_range("[a-z]+abc[Abc]*[1-5]"))
# print(concatenate("a+b*|c(5.f)"))
# print(infix_to_postfix("(A+.B*)?.(C|D)"))
print(to_postfix("."))        


(a|b|c|d|e|f|g|h|i|j|k|l|m|n|o|p|q|r|s|t|u|v|w|x|y|z|A|B|C|D|E|F|G|H|I|J|K|L|M|N|O|P|Q|R|S|T|U|V|W|X|Y|Z|0|1|2|3|4|5|6|7|8|9)
(a|b|c|d|e|f|g|h|i|j|k|l|m|n|o|p|q|r|s|t|u|v|w|x|y|z|A|B|C|D|E|F|G|H|I|J|K|L|M|N|O|P|Q|R|S|T|U|V|W|X|Y|Z|0|1|2|3|4|5|6|7|8|9)
(a|b|c|d|e|f|g|h|i|j|k|l|m|n|o|p|q|r|s|t|u|v|w|x|y|z|A|B|C|D|E|F|G|H|I|J|K|L|M|N|O|P|Q|R|S|T|U|V|W|X|Y|Z|0|1|2|3|4|5|6|7|8|9)
ab|c|d|e|f|g|h|i|j|k|l|m|n|o|p|q|r|s|t|u|v|w|x|y|z|A|B|C|D|E|F|G|H|I|J|K|L|M|N|O|P|Q|R|S|T|U|V|W|X|Y|Z|0|1|2|3|4|5|6|7|8|9|
ab|c|d|e|f|g|h|i|j|k|l|m|n|o|p|q|r|s|t|u|v|w|x|y|z|A|B|C|D|E|F|G|H|I|J|K|L|M|N|O|P|Q|R|S|T|U|V|W|X|Y|Z|0|1|2|3|4|5|6|7|8|9|


# Postfix to NFA

In [4]:
class edge:
    def __init__(self):
       self.label=None
       self.destination=None


class state:
       def __init__(self):
        self.label=None
        self.outgoing_edges=[]




class NFA:    
       def __init__(self, start, accept,states):
        self.start_state = start
        self.acceptance_state = accept
        self.states = states
        

def construct_nfa (label, id, stack):
    start = state()
    acceptance = state()
    start.label = "S"+str(id)
    acceptance.label = "S"+str(id+1)

    Edge = edge()
    Edge.label = label
    Edge.destination = acceptance 
    start.outgoing_edges.append(Edge)

    result_nfa = NFA(start, acceptance, [start,acceptance])
    stack.append(result_nfa)
    #print(result_nfa.inner_states)

    return result_nfa, id+2


def nfa_concatination (stack):
   
    
    nfa2 = stack.pop()
    nfa1 = stack.pop()


    Edge = edge()
    Edge.label = 'ε'
    Edge.destination = nfa2.start_state
    nfa1.acceptance_state.outgoing_edges.append(Edge)

    #print(nfa1)
    #print(nfa1.inner_states)

    resultnfa = NFA(nfa1.start_state,  nfa2.acceptance_state, nfa1.states + nfa2.states)
    stack.append(resultnfa)
    return resultnfa


def nfa_or (stack, id):
  nfa1 = stack.pop()
  nfa2 = stack.pop()
  newStart = state()
  newStart.label= "S"+str(id)
  newEnd = state()
  newEnd.label = "S"+str(id+1)

  Edge1 = edge()
  Edge1.label = 'ε'
  Edge1.destination = nfa1.start_state

  Edge2 = edge()
  Edge2.label = 'ε'
  Edge2.destination = nfa2.start_state

  newStart.outgoing_edges.append(Edge1)
  newStart.outgoing_edges.append(Edge2)


  Edge3 = edge()
  Edge3.label = 'ε'
  Edge3.destination = newEnd

  nfa1.acceptance_state.outgoing_edges.append(Edge3)

  Edge4 = edge()
  Edge4.label = 'ε'
  Edge4.destination = newEnd
  nfa2.acceptance_state.outgoing_edges.append(Edge4)


  result = NFA(newStart, newEnd, [newStart,newEnd]+ nfa1.states + nfa2.states)
  stack.append(result)
  return result, id+2



    
def nfa_json (nefa:NFA):
  outputJson = dict()
  outputJson["startingState"] = nefa.start_state.label
  for stat in nefa.states:
    stateDict = dict()
    if stat == nefa.acceptance_state:
      stateDict["isTerminatingState"] = True
    else:
      stateDict["isTerminatingState"] = False
    for edg in stat.outgoing_edges:
      if(edg.label in stateDict.keys()):
        stateDict[edg.label] = [] + [stateDict[edg.label]] + [edg.destination.label]
      else:
        stateDict[edg.label] = edg.destination.label
    outputJson[stat.label] = stateDict
  nfaOutFile = open('NFA.json', 'w')
  JsonObject = json.dump(outputJson, nfaOutFile,indent=6, ensure_ascii=False)
  nfaOutFile.close()
  return JsonObject

def postfix_to_nfa(postfix):
     stack = []
     id = 1

     for i in range(len(postfix)):
         if postfix[i] in alphanumeric:
           _ ,id =  construct_nfa(postfix[i],id,stack)
         elif postfix[i] == '.':   
             nfa_concatination(stack) 
         elif postfix[i] == '|':   
            _,id = nfa_or(stack,id)     
     result = stack.pop()
     return result







In [10]:
postfix = to_postfix("(a|b)")
nfa = postfix_to_nfa(postfix)
json_obj = nfa_json(nfa)


(a|b)
(a|b)
(a|b)
ab|


# NFA to DFA

In [20]:
def get_epsilon_closure(nfa, input_state):

    stack = [input_state]
    closure = set()

    while stack:

        current_state = stack.pop()
        closure.add(current_state)

        if "ε" in nfa[current_state]:

            next_states = nfa[current_state]["ε"]
            if isinstance(next_states, str):
                next_states = [next_states]

            for state in next_states:
                if state not in closure:
                    stack.append(state)
                    closure.add(state)

    return tuple(sorted(closure))

def convert_nfa_to_dfa(nfa):

    nfa_start_state = nfa["startingState"]
    dfa_start_state = get_epsilon_closure(nfa,nfa_start_state)

    dfa = {}
    dfa["startingState"] = dfa_start_state

    stack = [dfa_start_state]

    while stack:
        current_state = stack.pop()
        dfa[current_state] = {}     #current_state = (s0,s1,s2) 
        dfa[current_state]["isTerminatingState"] = False

        for state in current_state:
            nfa_state = nfa[state]   # nfa_state -> dic of s0 in nfa

            for sub_key in nfa_state:

                if sub_key == "isTerminatingState":
                    dfa[current_state]["isTerminatingState"] |= nfa_state["isTerminatingState"]

                elif sub_key != "ε": # edges of s0
                    next_states = get_epsilon_closure(nfa,nfa_state[sub_key])
                    if sub_key in dfa[current_state]: # if edge already in dfa state
                        temp_set= set(dfa[current_state][sub_key])
                        temp_set.update(next_states)
                        dfa[current_state][sub_key] = tuple(sorted(temp_set))
                    else:
                        dfa[current_state][sub_key] = next_states
        
        for new_sub_key in dfa[current_state]:
            if new_sub_key != "isTerminatingState":
                new_added_state = dfa[current_state][new_sub_key]
                if new_added_state not in dfa:
                    stack.append(new_added_state)

    return dfa

def get_final_dfa(dfa):

    # create a dict for states with their new names
    new_states_names = {}
    for i,state in enumerate(dfa):
        if state != "startingState":
            new_states_names[state] = "S"+str(i-1)
    print(new_states_names)

    # replace states with their new names
    new_dfa = dfa.copy()

    for key in dfa:
        if key != "startingState":

            for sub_key in dfa[key]:
                if sub_key !="isTerminatingState":
                    old_state = dfa[key][sub_key]  #('S1', 'S3')
                    new_state = new_states_names[old_state] #'S0'
                    new_dfa[key][sub_key] = new_state

            old_state = key  #('S1', 'S3')
            new_state = new_states_names[old_state] #'S0'
            new_dfa[new_state]= new_dfa[old_state]
            del new_dfa[key]

        else:
            old_state = dfa[key]
            new_state = new_states_names[old_state]
            new_dfa[key] = new_state

    return new_dfa

In [25]:
test_nfa= {
      "startingState": "S11",
      "S11": {
            "isTerminatingState": False,
            "ε": [
                  "S9",
                  "S12"
            ]
      },
      "S12": {
            "isTerminatingState": True
      },
      "S9": {
            "isTerminatingState": False,
            "ε": [
                  "S7",
                  "S5"
            ]
      },
      "S10": {
            "isTerminatingState": False,
            "ε": [
                  "S11",
                  "S12"
            ]
      },
      "S7": {
            "isTerminatingState": False,
            "c": "S8"
      },
      "S8": {
            "isTerminatingState": False,
            "ε": "S10"
      },
      "S5": {
            "isTerminatingState": False,
            "ε": [
                  "S3",
                  "S1"
            ]
      },
      "S6": {
            "isTerminatingState": False,
            "ε": "S10"
      },
      "S3": {
            "isTerminatingState": False,
            "b": "S4"
      },
      "S4": {
            "isTerminatingState": False,
            "ε": "S6"
      },
      "S1": {
            "isTerminatingState": False,
            "a": "S2"
      },
      "S2": {
            "isTerminatingState": False,
            "ε": "S6"
      }
}
test_dfa = convert_nfa_to_dfa(test_nfa)
print(test_dfa)
final_dfa = get_final_dfa (test_dfa)
print(final_dfa)


{'startingState': ('S1', 'S11', 'S12', 'S3', 'S5', 'S7', 'S9'), ('S1', 'S11', 'S12', 'S3', 'S5', 'S7', 'S9'): {'isTerminatingState': True, 'a': ('S1', 'S10', 'S11', 'S12', 'S2', 'S3', 'S5', 'S6', 'S7', 'S9'), 'b': ('S1', 'S10', 'S11', 'S12', 'S3', 'S4', 'S5', 'S6', 'S7', 'S9'), 'c': ('S1', 'S10', 'S11', 'S12', 'S3', 'S5', 'S7', 'S8', 'S9')}, ('S1', 'S10', 'S11', 'S12', 'S3', 'S5', 'S7', 'S8', 'S9'): {'isTerminatingState': True, 'a': ('S1', 'S10', 'S11', 'S12', 'S2', 'S3', 'S5', 'S6', 'S7', 'S9'), 'b': ('S1', 'S10', 'S11', 'S12', 'S3', 'S4', 'S5', 'S6', 'S7', 'S9'), 'c': ('S1', 'S10', 'S11', 'S12', 'S3', 'S5', 'S7', 'S8', 'S9')}, ('S1', 'S10', 'S11', 'S12', 'S3', 'S4', 'S5', 'S6', 'S7', 'S9'): {'isTerminatingState': True, 'a': ('S1', 'S10', 'S11', 'S12', 'S2', 'S3', 'S5', 'S6', 'S7', 'S9'), 'b': ('S1', 'S10', 'S11', 'S12', 'S3', 'S4', 'S5', 'S6', 'S7', 'S9'), 'c': ('S1', 'S10', 'S11', 'S12', 'S3', 'S5', 'S7', 'S8', 'S9')}, ('S1', 'S10', 'S11', 'S12', 'S2', 'S3', 'S5', 'S6', 'S7', 'S9'):

# Minimized DFA

In [77]:
def minimize_dfa(dfa):

    unique_inputs = set()

    groups_dic = {}
    groups_dic[1] = set()
    groups_dic[2] = set()

    for key in dfa:
        if key != "startingState":
            if dfa[key]["isTerminatingState"]:
                groups_dic[1].add(key)
            else:
                groups_dic[2].add(key)
        
            for sub_key in dfa[key]:
                if sub_key != "isTerminatingState":
                    unique_inputs.add(sub_key)

    if len(groups_dic[1]) == 0:
        del groups_dic[1]

    if len(groups_dic[2]) == 0:
        del groups_dic[2]

    print(groups_dic,unique_inputs)



    stop = False
    new_dic={}

    is_group_divided = False
    to_delete = None

    groups_count = len(groups_dic)

    print(groups_count)
    no_div_count = 0
    input_counter = 0
    while not stop:
        if (no_div_count == groups_count):
                print('+++++++++++++++++++++++',no_div_count)
                stop = True
            
      
        
        for group_id in groups_dic.keys():
           
            if is_group_divided:
                print('-------------')
                for k in new_dic:
                    groups_dic[k]=new_dic[k]

                if to_delete in groups_dic.keys():
                    del groups_dic[to_delete]
                print(groups_dic)
                groups_count = len(groups_dic)
                # input_counter = 0
                is_group_divided =  False

                # if (no_div_count == groups_count):
                #         print('+++++++++++++++++++++++',no_div_count)
                #         stop = True
                break
            
            states_in_group = groups_dic[group_id]
                        
            input_counter = 0
            for input in unique_inputs:
                input_counter += 1

                print(input)

                dic={}
                for state in states_in_group:
                    if input in dfa[state]:
                        next_state = dfa[state][input]

                        for group, state_list in groups_dic.items():
                            if next_state in state_list:
                                if group in dic.keys():
                                    dic[group].add(state)
                                else:
                                    dic[group]={state}
                                # break
                    else: 
                        if "-" in dic.keys():
                            dic["-"].add(state)
                        else:
                            dic["-"]={state}
                print(dic)   

                new_dic={}
                i = max(groups_dic.keys()) + 1
                for new_group in dic.keys():
                        new_dic[i]=dic[new_group]
                        i+=1
                print(new_dic)

                if (len(new_dic) > 1): 
                    print('======================')
                    is_group_divided = True
                    to_delete = group_id
                    break
                else:
                    is_group_divided = False
                    no_div_count += 1
                    print('ooooooooooooooooooooooooo',no_div_count)
                    if(no_div_count >  groups_count*len(unique_inputs)):
                        stop = True
                        
            
                    
                      
                    # if (no_div_count == groups_count):
                    #     print('+++++++++++++++++++++++',no_div_count)
                    #     stop = True
                    
                    # stop = True
                    # if (no_div_count == groups_count):
                    #     print('+++++++++++++++++++++++',no_div_count)
                    #     stop = True
            
            # if (no_div_count == groups_count):
            #             print('+++++++++++++++++++++++',no_div_count)
            #             stop = True

                # del groups_dic[group_id]
                    
            # if (no_div_count == groups_count):
            #     print('+++++++++++++++++++++++',no_div_count)
            #     stop = True



                        
                    

In [79]:
test_for_min={'startingState': 'S0', 'S0': {'isTerminatingState': False, 'b': 'S4', 'c': 'S1'}, 'S1': {'isTerminatingState': False, 'b': 'S4', 'd': 'S2', 'c': 'S1'}, 'S2': {'isTerminatingState': True, 'c': 'S3'}, 'S3': {'isTerminatingState': False, 'd': 'S2'}, 'S4': {'isTerminatingState': False, 'b': 'S4','c':'S1'}}

minimize_dfa (test_for_min)
# minimize_dfa (final_dfa)

{1: {'S2'}, 2: {'S1', 'S3', 'S4', 'S0'}} {'d', 'b', 'c'}
2
d
{'-': {'S2'}}
{3: {'S2'}}
ooooooooooooooooooooooooo 1
b
{'-': {'S2'}}
{3: {'S2'}}
ooooooooooooooooooooooooo 2
c
{2: {'S2'}}
{3: {'S2'}}
ooooooooooooooooooooooooo 3
d
{1: {'S3', 'S1'}, '-': {'S4', 'S0'}}
{3: {'S3', 'S1'}, 4: {'S4', 'S0'}}
-------------
{1: {'S2'}, 3: {'S3', 'S1'}, 4: {'S4', 'S0'}}
+++++++++++++++++++++++ 3
d
{'-': {'S2'}}
{5: {'S2'}}
ooooooooooooooooooooooooo 4
b
{'-': {'S2'}}
{5: {'S2'}}
ooooooooooooooooooooooooo 5
c
{3: {'S2'}}
{5: {'S2'}}
ooooooooooooooooooooooooo 6
d
{1: {'S3', 'S1'}}
{5: {'S3', 'S1'}}
ooooooooooooooooooooooooo 7
b
{'-': {'S3'}, 4: {'S1'}}
{5: {'S3'}, 6: {'S1'}}
-------------
{1: {'S2'}, 4: {'S4', 'S0'}, 5: {'S3'}, 6: {'S1'}}
